In [ ]:
# !pip install imageio

In [ ]:
# !python train.py -h

In [ ]:
# files modified
"""
Pluralistic/train.py
Pluralistic/options/base_options.py
"""
--continue_train

In [ ]:
!python train.py --name celeba_random --img_file linmao-test_txt/small_set --niter 1000 --save_iters_freq 1000 --save_latest_freq 10 --batchSize 4 --continue_train


--------------Options--------------
batchSize: 4
checkpoints_dir: ./checkpoints
continue_train: True
display_freq: 100
display_id: 1
display_port: 8097
display_single_pane_ncols: 0
display_winsize: 256
fineSize: [512, 512]
gan_mode: lsgan
gpu_ids: 0
img_file: linmao-test_txt/small_set
isTrain: True
iter_count: 1
lambda_g: 1.0
lambda_kl: 20.0
lambda_rec: 20.0
loadSize: [522, 522]
lr: 0.0001
lr_policy: lambda
mask_file: none
mask_type: [1, 2]
model: pluralistic
nThreads: 8
name: celeba_random
niter: 1000
niter_decay: 0
no_augment: False
no_flip: False
no_html: False
no_rotation: False
no_shuffle: False
output_scale: 4
print_freq: 100
resize_or_crop: resize_and_crop
save_iters_freq: 1000
save_latest_freq: 10
train_paths: two
which_iter: latest
----------------End----------------
training images = 16
ResEncoder(
  (block0): ResBlockEncoderOptimized(
    (conv1): SpectralNorm(
      (module): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (conv2): SpectralNorm(
 

In [ ]:
# !python test.py -h

In [ ]:
!python test.py  --name celeba_random --img_file datasets/place2